## Library imports

In [ ]:
from premise import *
import bw2data as bd
import bw2io as bo
import xarray as xr

## Data imports

In [ ]:
coord = pd.read_csv("Coordinate_unique.csv")

In [ ]:
DAC_inputs = {
    "Electricity": {
        2020: 0.345,
        2025: 0.319,
        2030: 0.311,
        2035: 0.307,
        2040: 0.304,
        2045: 0.301,
        2050: 0.299
    },
    "Heat": {
        2020: 6.28,
        2025: 5.81,
        2030: 5.67,
        2035: 5.58,
        2040: 5.53,
        2045: 5.48,
        2050: 5.44
    },

}

# Convert dictionaries into an xarray Dataset
years = list(DAC_inputs['Electricity'].keys())
electricity_values = list(DAC_inputs['Electricity'].values())
heat_values = list(DAC_inputs['Heat'].values())

dataset = xr.Dataset({
    'Electricity': ('Year', electricity_values),
    'Heat': ('Year', heat_values),
}, coords={'Year': years})

## Brightway project set up

In [ ]:
bd.projects.set_current("MPTEX_demo")

In [ ]:
bo.bw2setup()

In [ ]:
bd.databases

In [ ]:
# file path to the place with the ecoinvent spold files (datasets directory)
fpei39 = "/etc/data/ecospold/datasets"
if 'ecoinvent 3.9.1 cutoff' in bd.databases:
    print("Database has already been imported")
else:
    ei39 = bo.SingleOutputEcospold2Importer(fpei39, 'ecoinvent 3.9.1 cutoff')
    ei39.apply_strategies()
    ei39.statistics()
    
    if len(list(ei39.unlinked)) == 0:
        ei39.write_database()

### Create the desired premise scenarios

In [ ]:
ndb = NewDatabase(
    scenarios=[
        {"model":"image", "pathway":"SSP2-RCP19", "year":2029},
        #{"model":"image", "pathway":"SSP2-RCP19", "year":2030},
        #{"model":"image", "pathway":"SSP2-RCP19", "year":2050},
        #{"model":"remind","pathway":"SSP2-Base","year":2039}
    ],
    source_db="ecoinvent 3.9.1 cutoff", # <-- name of the database in the BW2 project. Must be a string.
    source_version="3.9", # <-- version of ecoinvent. Can be "3.5", "3.6", "3.7" or "3.8". Must be a string.
    key='tUePmX_S5B8ieZkkM7WUU2CnO8SmShwmAeWK9x2rTFo=' # <-- decryption key
    # to be requested from the library maintainers if you want ot use default scenarios included in `premise`
)

In [ ]:
ndb.update()

In [ ]:
#Save as a database
ndb.write_db_to_brightway(name=["image-SSP2-RCP19-2029",])

## User input variables

In [ ]:
db_name = 'image-SSP2-RCP19-2029'
year = 2029
DAC_ds_name = 'carbon dioxide, captured from atmosphere, with a solvent-based direct air capture system, 1MtCO2'
LCIA_method = ('IPCC 2021', 'climate change: biogenic', 'global warming potential (GWP100)')

### Update PV data

In [ ]:
pv_datasets = pv_market_mixes_identifier(db_name)    
display(pv_datasets)

In [ ]:
pv_updated_ds = add_custom_production_data_to_df(pv_datasets, generate_pv_prod_data(coord))

In [ ]:
pv_datasets1 = update_pv_efficiencies(db_name,pv_updated_ds)
display(pv_datasets1)

### Update DAC datasets

In [ ]:
update_DAC_efficiencies(db_name,year)
#generate_new_DAC_inventories(db_name)

### Run an LCA

In [ ]:
DAC_list = []

# Get a DAC dataset
for ds in bd.Database(db_name): 
    if ds['name'] == DAC_ds_name:
        DAC_list.append((db_name,ds['code']))

In [ ]:
lca = bc.LCA(demand={DAC_list[0]: 1}, method=LCIA_method)
lca.lci()
lca.lcia()
lca.score